In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Multiple Linear Regression
* Y와 선형 관계를 맺고 있는 X 변수들이 여러개일 때 사용
* Simple Linear Regression과 fitted value와 잔차의 개념 동일하며, 똑같이 LSM을 사용하여 모델을 학습시킴(model fitting)
* 어떤 X 변수의 계수 크기가 클수록,   
X 변수가 Y 변수에 미치는 영향력이 크다 / X가 Y를 많이 설명한다 /   
X와 Y가 관련성이 깊다고 해석함

### Assessing the Model 모델 성능 평가 지표
* 평균 제곱근 오차 (RMSE; root mean squared error):   
잔차 제곱의 평균에 제곱근을 취한 값. 낮을수록 모델 성능이 좋다고 판단


* 결정계수 (coef of determination, R-squared)
    * Y가 가지는 전체 변동(total variability) 중 모델(set of Xs)가 설명할 수 있는 정도를 나타냄
    * 0~1까지 값
    * 높을수록 모델 성능이 좋다고 판단


* 어떤 변수를 regression model 넣을지 판단에 사용하는 용도로,   
해당 변수의 t-statistic과 그 p-value를 사용할 수 있음.   
t-statistic이 높고 p-value 낮을수록 해당 변수가 y랑 관련성이 높다는 뜻으로 해석할 수 있음   
즉. 회귀 모델에 넣는 것이 좋다고 판단함

In [ ]:
# Multiple Linear Regression 모델 구축 및 학습
# 구축한 모델로부터 모델 성능 평가 지표 도출 및 모델 판단하기

In [ ]:
# 데이터 읽기

df_train = pd.read_csv('empty_train.csv')
df_test = pd.read_csv('empty_test.csv')

df_train.head() # 독립 변수: 각 시군별 통계 수치 / 종속 변수: 빈집자수

,Row Labels,연도,도시지역면적,비만율,유치원 원아수,초등학교 학생수,토지거래면적,혼인건수,노인 천명당 노인여가복지시설수,1인가구비율,...,대학교수,독거노인가구비율,유치원수,인구 십만명당 문화기반시설수,인구 천명당 사설학원수,주택수,초등학교수,평균연령,빈집수,빈집지수
0,강원 화천군,2016,13230291,33.8,378,1159,7131,190,11.61,43.1,...,0,9.3,14,26.65,0.76,9349,13,42.5,209,2.186650
1,충북 영동군,2017,31106651,27.8,417,1745,15329,166,18.34,34.6,...,1,15.7,16,17.91,0.84,20925,15,49.1,2657,11.267068
2,전남 해남군,2018,57390376,37.1,534,2838,29088,284,21.04,34.4,...,0,20.0,22,13.73,0.89,30404,20,50.3,4162,12.040734
3,경북 구미시,2016,185781767,27.2,10366,26891,15345,2618,8.03,30.2,...,3,3.8,99,2.62,2.01,133737,48,36.4,10869,7.516286
4,인천 강화군,2016,17306615,36.7,617,2495,9819,219,8.87,27.8,...,1,13.1,21,14.70,0.74,26660,22,50.6,3495,11.590118


In [ ]:
# 데이터 전처리

X_train, y_train = df_train.iloc[:, 1:-1], df_train['빈집지수']

In [ ]:
Xs = sm.add_constant(X_train)
rm = sm.OLS(y_train, Xs).fit()

rm.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   빈집지수   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     99.82
Date:                Mon, 16 May 2022   Prob (F-statistic):          2.29e-186
Time:                        00:16:49   Log-Likelihood:                -1301.3
No. Observations:                 615   AIC:                             2651.
Df Residuals:                     591   BIC:                             2757.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             -160.9451    290.220     -0.555      0.579    -730.933     409.042
연도                   0.0783      0.144      0.544      0.587      -0.205       0.361
도시지역면적           -1.975e-09   1.46e-09     -1.353      0.177   -4.84e-09    8.92e-10
비만율                  0.0568      0.029      1.981      0.048       0.000       0.113
유치원 원아수              0.0002      0.000      1.253      0.211      -0.000       0.000
초등학교 학생수             0.0002   6.32e-05      2.505      0.013    3.42e-05       0.000
토지거래면적            7.471e-06   1.94e-05      0.386      0.700   -3.06e-05    4.55e-05
혼인건수                 0.0005      0.001      0.615      0.539      -0.001       0.002
노인 천명당 노인여가복지시설수     0.0704      0.031      2.257      0.024       0.009       0.132
1인가구비율              -0.0510      0.030     -1.707      0.088      -0.110       0.008
등록외국인 현황         -2.682e-05   1.93e-05     -1.386      0.166   -6.48e-05    1.12e-05
유아 천명당 보육시설수         0.0638      0.032      2.004      0.045       0.001       0.126
지가변동률               -0.1390      0.070     -1.979      0.048      -0.277      -0.001
출생아수                 0.0004      0.001      0.657      0.512      -0.001       0.001
학급당 학생수             -0.0330      0.068     -0.483      0.629      -0.167       0.101
대학교수                 0.0112      0.068      0.165      0.869      -0.122       0.144
독거노인가구비율             0.2309      0.085      2.721      0.007       0.064       0.398
유치원수                -0.0017      0.015     -0.115      0.908      -0.031       0.028
인구 십만명당 문화기반시설수     -0.0071      0.013     -0.533      0.594      -0.033       0.019
인구 천명당 사설학원수         0.3202      0.227      1.410      0.159      -0.126       0.766
주택수              -8.071e-05   1.12e-05     -7.239      0.000      -0.000   -5.88e-05
초등학교수               -0.0948      0.024     -3.925      0.000      -0.142      -0.047
평균연령                 0.1982      0.095      2.084      0.038       0.011       0.385
빈집수                  0.0009   3.76e-05     23.484      0.000       0.001       0.001
==============================================================================
Omnibus:                      216.295   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1845.009
Skew:                           1.313   Prob(JB):                         0.00
Kurtosis:                      11.069   Cond. No.                     4.20e+11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.2e+11. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# 계수의 의미가 이상한 사례 확인
# 다중 공선성 확인(독립 변수간 산점도, VIF)
# 모델 변형 및 결과 확인

# 다중공선성 파악하는 방법
# 1. 독립변수 간에 산점도 / corr
# 2. VIF

In [ ]:
# VIF: 어떤 독립 변수가 다른 독립 변수들과 선형 관계를 가지는지 확인할 수 있는 지표

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(Xs.values, i) for i in range(Xs.shape[1])]
vif['predictor'] = Xs.columns
vif # 5 이상 (유의) / 10 이상 (다중공선성 있다고 판단 - 변수 제거하여 사용)

,VIF,predictor
0,1.234879e+07,const
1,2.027149e+00,연도
2,2.682574e+00,도시지역면적
3,1.655504e+00,비만율
4,3.809234e+01,유치원 원아수
5,9.376837e+01,초등학교 학생수
6,3.712360e+00,토지거래면적
7,1.290803e+02,혼인건수
8,8.595411e+00,노인 천명당 노인여가복지시설수
9,3.490507e+00,1인가구비율


In [ ]:
# 모델 학습

sc = StandardScaler() # (X - m) / std
X_train_sc = sc.fit_transform(X_train)

rm = LinearRegression().fit(X_train_sc, y_train)
print(rm.coef_, rm.intercept_)

[ 0.06394545 -0.18304501  0.21054134  0.63848662  2.00324291  0.06140159
  0.5765919   0.54641189 -0.26336661 -0.19972942  0.21464247 -0.20534407
  0.61547055 -0.14407682  0.0203193   1.33611235 -0.05915995 -0.065443
  0.18497826 -5.42477602 -1.75821031  0.94154053  4.16710094] 8.812093075447155


In [ ]:
fitted = rm.predict(X_train_sc)

print('RMSE: ', np.sqrt(mean_squared_error(y_train, fitted)))
print('R2 score: ', r2_score(y_train, fitted))

RMSE:  2.007745024002373
R2 score:  0.795280140637058


In [ ]:
# 다중공선성 있다고 판단 - 변수 제거하여 사용

X_train = X_train.drop(['연도',
                        '도시지역면적',
                        '비만율',
                        '혼인건수',
                        '1인가구비율',
                        '지가변동률',
                        '인구 십만명당 문화기반시설수',], axis=1)

In [ ]:
Xs = sm.add_constant(X_train)
rm = sm.OLS(y_train, Xs).fit()

rm.summary()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   빈집지수   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     140.5
Date:                Mon, 16 May 2022   Prob (F-statistic):          2.38e-190
Time:                        00:16:49   Log-Likelihood:                -1309.3
No. Observations:                 615   AIC:                             2653.
Df Residuals:                     598   BIC:                             2728.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -5.5229      4.358     -1.267      0.206     -14.082       3.036
유치원 원아수              0.0001      0.000      0.905      0.366      -0.000       0.000
초등학교 학생수             0.0002   5.78e-05      3.087      0.002    6.49e-05       0.000
토지거래면적            -5.09e-06   1.86e-05     -0.273      0.785   -4.17e-05    3.15e-05
노인 천명당 노인여가복지시설수     0.0882      0.030      2.974      0.003       0.030       0.146
등록외국인 현황         -2.791e-05    1.6e-05     -1.745      0.082   -5.93e-05     3.5e-06
유아 천명당 보육시설수         0.0993      0.030      3.289      0.001       0.040       0.159
출생아수                 0.0006      0.000      2.232      0.026    6.71e-05       0.001
학급당 학생수             -0.0199      0.057     -0.351      0.726      -0.131       0.092
대학교수                -0.0621      0.061     -1.018      0.309      -0.182       0.058
독거노인가구비율             0.1711      0.079      2.155      0.032       0.015       0.327
유치원수                 0.0059      0.015      0.398      0.691      -0.023       0.035
인구 천명당 사설학원수         0.1669      0.218      0.765      0.445      -0.262       0.595
주택수               -8.07e-05   8.99e-06     -8.979      0.000   -9.84e-05   -6.31e-05
초등학교수               -0.0982      0.024     -4.167      0.000      -0.144      -0.052
평균연령                 0.2430      0.092      2.628      0.009       0.061       0.425
빈집수                  0.0009   3.59e-05     24.626      0.000       0.001       0.001
==============================================================================
Omnibus:                      232.269   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2217.027
Skew:                           1.397   Prob(JB):                         0.00
Kurtosis:                      11.872   Cond. No.                     5.39e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.39e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# VIF: 어떤 독립 변수가 다른 독립 변수들과 선형 관계를 가지는지 확인할 수 있는 지표

vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(Xs.values, i) for i in range(Xs.shape[1])]
vif['predictor'] = Xs.columns
vif # 5 이상 (유의) / 10 이상 (다중공선성 있다고 판단 - 변수 제거하여 사용)

,VIF,predictor
0,2745.190448,const
1,35.458738,유치원 원아수
2,77.163498,초등학교 학생수
3,3.392898,토지거래면적
4,7.654699,노인 천명당 노인여가복지시설수
5,2.050392,등록외국인 현황
6,1.489171,유아 천명당 보육시설수
7,26.955815,출생아수
8,8.868678,학급당 학생수
9,1.773344,대학교수


In [ ]:
# 모델 학습

rm = LinearRegression().fit(X_train, y_train)
print(rm.coef_, rm.intercept_)

[ 1.27207568e-04  1.78330923e-04 -5.09026104e-06  8.82200869e-02
 -2.79051048e-05  9.92833810e-02  5.59571839e-04 -1.99023655e-02
 -6.20799139e-02  1.71067244e-01  5.87055850e-03  1.66915820e-01
 -8.07032650e-05 -9.81573287e-02  2.42967642e-01  8.83612721e-04] -5.522873407654481


In [ ]:
fitted = rm.predict(X_train)

print('RMSE: ', np.sqrt(mean_squared_error(y_train, fitted)))
print('R2 score: ', r2_score(y_train, fitted))

RMSE:  2.0340910388153843
R2 score:  0.7898721431108797


In [ ]:
# ~~~~~~~~~~~~~

X_train, y_train = df_train.iloc[:, 1:-1], df_train['빈집지수']

sc = StandardScaler() # (X - m) / std
X_train_sc = sc.fit_transform(X_train)

rm = LinearRegression().fit(X_train_sc, y_train)

fitted = rm.predict(X_train_sc)

print('RMSE: ', np.sqrt(mean_squared_error(y_train, fitted)))
print('R2 score: ', r2_score(y_train, fitted))

RMSE:  2.007745024002373
R2 score:  0.795280140637058


In [ ]:
### 학습한 모델을 테스트 데이터셋에 적용
# (1) 테스트 데이터셋에, 트레인 데이터셋과 동일하게 전처리 등 적용
X_test = df_test.iloc[:, 1:]
X_test_sc = sc.fit_transform(X_test)

# (2) 모델 수행 결과 기록
y_pred = rm.predict(X_test_sc)

# (3) 결과를 df_test에 추가
df_test['빈집지수'] = y_pred

In [ ]:
df_test.head()

,Row Labels,연도,도시지역면적,비만율,유치원 원아수,초등학교 학생수,토지거래면적,혼인건수,노인 천명당 노인여가복지시설수,1인가구비율,...,대학교수,독거노인가구비율,유치원수,인구 십만명당 문화기반시설수,인구 천명당 사설학원수,주택수,초등학교수,평균연령,빈집수,빈집지수
0,경기 여주시,2018,26093192,33.1,1191,5526,14607,484,10.34,28.4,...,1,8.8,34,21.40,1.08,40237,23,44.7,4273,9.270244
1,서울 강북구,2016,23635959,28.9,2105,11225,501,1500,1.47,29.2,...,0,7.6,24,3.36,0.71,93623,14,43.3,3046,4.442795
2,대구 수성구,2016,76466396,20.2,6469,24717,1625,1572,3.03,21.3,...,1,6.1,55,3.13,2.12,133183,34,40.7,5412,3.643867
3,전남 광양시,2016,176886009,26.4,1697,8947,8611,809,12.97,25.8,...,2,6.3,39,4.50,1.67,59402,28,39.3,9333,12.030948
4,대구 서구,2017,17328303,27.2,1403,6161,487,771,1.76,30.8,...,0,9.5,24,2.60,0.86,54424,17,45.5,1921,5.557448


In [ ]:
# (4) 결과를 파일로 저장
# submit.csv 이름으로 df_test 저장
df_test.to_csv('./answer_지상준.csv', index=False, encoding='utf-8-sig') # 한글 저장

In [ ]:
# 스케일러 및 모델 저장
import joblib

joblib.dump(sc, 'house_scaler.pkl')
joblib.dump(rm, 'house_log_model.pkl')

['house_log_model.pkl']

In [ ]:
# 모델 load 시 같은 결과가 나오는지 확인
scaler = joblib.load('house_scaler.pkl')
model = joblib.load('house_log_model.pkl')

y_pred_re = model.predict(X_test_sc)
print(y_pred == y_pred_re)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]
